In [91]:
import pandas as pd
import networkx as nx
import heapq

In [111]:
def custom_dijkstra(G, source, threshold):
    # Initialize distances and visited set
    distances = {node: float('inf') for node in G.nodes}
    distances[source] = 0
    visited = set()

    # Priority queue (heap) to store nodes and their distances
    pq = [(0, source)]

    while pq:
        current_distance, current_node = heapq.heappop(pq)

        # Stop if all nodes within 15 have been visited
        if current_distance > threshold:
            break

        if current_node not in visited:
            visited.add(current_node)

        for neighbor, edge_data in G[current_node].items():
            weight = edge_data[0]['weight']
            new_distance = current_distance + weight
            if new_distance < distances[neighbor]:
                distances[neighbor] = new_distance
                heapq.heappush(pq, (new_distance, neighbor))

    return visited

https://geoffboeing.com/2016/11/osmnx-python-street-networks/

I have 2 csv files 'edges' and 'nodes' where edges contain IDs for 'source' and 'target' and also 'weight'. nodes contains the 'id' and also 'label' which could be a string or is empty.

I want to load this into Python in a graph data structure with networkx as a MultiGraph which is undirected. I then want to run Dijkstra from each node with a non-empty label and stop once all edges within weight of 15 have been searched from the source nodes.

In [112]:
# Read the data from the file
nodes = pd.read_csv('./min_city/data/nodes.csv',keep_default_na=False)
edges = pd.read_csv('./min_city/data/edges.csv')

In [113]:
# set of unique service
labels = nodes['label'].unique().tolist()
labels = [x for x in labels if len(x) > 0]

In [116]:
labels

['cafe']

In [114]:
# Create a MultiGraph from the edges DataFrame
G = nx.from_pandas_edgelist(edges, source='source', target='target', edge_attr='weight', create_using=nx.MultiGraph())

In [77]:
# Filter nodes with non-empty labels
labeled_nodes = nodes[nodes['label'] == labels[0]]

In [78]:
labeled_nodes

,id,label
4,4,cafe
6,6,cafe
7,7,cafe
23,23,cafe
25,25,cafe
26,26,cafe
27,27,cafe
28,28,cafe
29,29,cafe
32,32,cafe


In [119]:
FMC = set(G.nodes())
for idx, type in enumerate(labels):
    new_node_id = max(G.nodes) + idx + 1
    G.add_node(new_node_id) # create new node
    G.add_weighted_edges_from([(new_node_id, node, 0) for node in labeled_nodes['id']])
    FMC_tmp = custom_dijkstra(G, new_node_id, 15*10**6)
    G.remove_node(new_node_id)
    FMC = FMC.intersection(FMC_tmp)

In [120]:
FMC

{4, 6, 7, 23, 25, 26, 27, 28, 29, 32, 33}